In [2]:
!pip install twython


In [3]:
!git clone https://github.com/EnyanDai/FakeHealth.git


Cloning into 'FakeHealth'...
remote: Enumerating objects: 2292, done.
remote: Counting objects: 100% (2292/2292), done.
remote: Compressing objects: 100% (2211/2211), done.
remote: Total 2292 (delta 79), reused 2292 (delta 79), pack-reused 0 (from 0)
Receiving objects: 100% (2292/2292), 15.38 MiB | 11.85 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [5]:
!curl -O https://zenodo.org/record/4282522/files/FakeHealth.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   261  100   261    0     0    459      0 --:--:-- --:--:-- --:--:--   459


In [6]:
!unzip FakeHealth.zip -d FakeHealth


Archive:  FakeHealth.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of FakeHealth.zip or
        FakeHealth.zip.zip, and cannot find FakeHealth.zip.ZIP, period.


In [7]:
!ls FakeHealth


API  dataset  README.md


In [8]:
!ls FakeHealth/*


FakeHealth/README.md

FakeHealth/API:
collect_tweets.py  crawl_friends_profiles.py  news.py	 resource_server
collect_users.py   main.py		      resources  util

FakeHealth/dataset:
content  engagements  reviews


In [9]:
!ls FakeHealth/API

collect_tweets.py  crawl_friends_profiles.py  news.py	 resource_server
collect_users.py   main.py		      resources  util


In [10]:
!find FakeHealth -type f


FakeHealth/.git/refs/heads/master
FakeHealth/.git/refs/remotes/origin/HEAD
FakeHealth/.git/config
FakeHealth/.git/index
FakeHealth/.git/logs/refs/heads/master
FakeHealth/.git/logs/refs/remotes/origin/HEAD
FakeHealth/.git/logs/HEAD
FakeHealth/.git/description
FakeHealth/.git/hooks/pre-commit.sample
FakeHealth/.git/hooks/applypatch-msg.sample
FakeHealth/.git/hooks/push-to-checkout.sample
FakeHealth/.git/hooks/fsmonitor-watchman.sample
FakeHealth/.git/hooks/pre-applypatch.sample
FakeHealth/.git/hooks/pre-receive.sample
FakeHealth/.git/hooks/update.sample
FakeHealth/.git/hooks/prepare-commit-msg.sample
FakeHealth/.git/hooks/pre-merge-commit.sample
FakeHealth/.git/hooks/commit-msg.sample
FakeHealth/.git/hooks/post-update.sample
FakeHealth/.git/hooks/pre-push.sample
FakeHealth/.git/hooks/pre-rebase.sample
FakeHealth/.git/objects/pack/pack-eeb0dba1b2c184b91ade30615f2c189c5c607983.idx
FakeHealth/.git/objects/pack/pack-eeb0dba1b2c184b91ade30615f2c189c5c607983.pack
FakeHealth/.git/info/exclude
F

In [11]:
import json

with open("FakeHealth/dataset/reviews/HealthStory.json", "r") as f:
    health_story_data = json.load(f)

print(health_story_data[:5])  # Show first 5 entries


[{'link': 'https://www.healthnewsreview.org/review/guardian-jumps-to-conclusions-about-an-alzheimers-study-that-hasnt-even-begun/', 'title': 'Guardian jumps to conclusions about an Alzheimer’s study that hasn’t even begun', 'description': 'The story’s headline states: “Virtual reality to help detect early risk of Alzheimer’s.” That is misleading. The researchers haven’t even recruited study participants yet, much less conducted the study or analyzed the results.', 'original_title': 'Virtual reality to help detect early risk of Alzheimer’s', 'news_id': 'story_reviews_00000', 'rating': 1, 'reviewers': ['Matt Shipman', 'Doug Campos-Outcalt, MD, MPA', 'Joy Victory'], 'category': 'The Guardian', 'tags': ["alzheimer's disease"], 'source_link': 'https://www.theguardian.com/society/2018/dec/16/alzheimers-dementia-cure-virtual-reality-navigation-skills', 'summary': {'Our Review Summary': 'The story focuses on a forthcoming study that aims to evaluate if virtual reality technology can be used to

In [12]:
import pandas as pd

df = pd.DataFrame(health_story_data)
print(df.head())  # Show first few rows


                                                link  \
0  https://www.healthnewsreview.org/review/guardi...   
1  https://www.healthnewsreview.org/review/in-sto...   
2  https://www.healthnewsreview.org/review/using-...   
3  https://www.healthnewsreview.org/review/hard-t...   
4  https://www.healthnewsreview.org/review/a-litt...   

                                               title  \
0  Guardian jumps to conclusions about an Alzheim...   
1  In story on experimental contraceptive gel, Re...   
2  Using a single patient anecdote, ABC News hail...   
3  Hard to find the journalism in this brief Reut...   
4  A little more context would have boosted alrea...   

                                         description  \
0  The story’s headline states: “Virtual reality ...   
1  This leaves both investors and lay audiences w...   
2  Breakthrough? Based on the story of one patien...   
3  The story mostly just rehashes a drug company ...   
4  Did the study findings really earn a label 

In [13]:
df = df.dropna()


In [14]:
df['title'] = df['title'].astype(str).str.lower()


In [15]:
import re

def clean_text(title):
    text = re.sub(r"http\S+", "", title)  # Remove URLs
    text = re.sub(r"[^a-zA-Z0-9\s]", "", title)  # Remove special characters
    return text.strip()

df['title'] = df['title'].apply(clean_text)


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)  # Convert text to numbers
X = vectorizer.fit_transform(df['title'])  # Features (independent variable)
y = df['rating']  # Target (dependent variable)


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")


Accuracy: 0.33


In [18]:
!pip install xgboost


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Load dataset (assuming df already exists)
df['text'] = df['title'] + " " + df['description']  # Combine title & description for better context

# Convert labels: Assuming "rating" is the label, use 0 for fake, 1 for real
df['label'] = df['rating'].apply(lambda x: 0 if x <= 2 else 1)

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Convert text to numerical features using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Use max 5000 features for performance
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [20]:
# Train an XGBoost model
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
xgb_model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ XGBoost Accuracy: {accuracy:.2f}")



✅ XGBoost Accuracy: 0.74


In [21]:
import joblib

# Save the trained XGBoost model
joblib.dump(xgb_model, "xgboost_tfidf_model.pkl")
print("✅ XGBoost model saved as 'xgboost_tfidf_model.pkl'")


✅ XGBoost model saved as 'xgboost_tfidf_model.pkl'


In [22]:
# Load the trained XGBoost model
xgb_model = joblib.load("xgboost_tfidf_model.pkl")
print("✅ XGBoost model loaded successfully!")


✅ XGBoost model loaded successfully!


In [23]:
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


In [24]:
# Load XGBoost model
model = joblib.load("xgboost_tfidf_model.pkl")
print("✅ Model loaded successfully!")


✅ Model loaded successfully!


In [25]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Load JSON data
with open("FakeHealth/dataset/reviews/HealthStory.json", "r") as f:
    data = json.load(f)

# Convert JSON to DataFrame
df = pd.DataFrame(data)

# Check available columns
print(df.columns)

# Ensure 'text' column exists (if not, replace with the correct field)
if 'title' not in df.columns:
    raise KeyError("Column 'title' not found! Check your JSON structure.")

# Initialize and fit the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Adjust features if needed
X_tfidf = tfidf_vectorizer.fit_transform(df['title'])  # Transform the text data

# Save the fitted vectorizer
joblib.dump(tfidf_vectorizer, "tfidf_vectorizer.pkl")
print("✅ TF-IDF Vectorizer saved successfully!")


Index(['link', 'title', 'description', 'original_title', 'news_id', 'rating',
       'reviewers', 'category', 'tags', 'source_link', 'summary', 'criteria',
       'news_source'],
      dtype='object')
✅ TF-IDF Vectorizer saved successfully!


In [26]:
# Load the TF-IDF Vectorizer
tfidf_vectorizer = joblib.load("tfidf_vectorizer.pkl")
print("✅ TF-IDF Vectorizer loaded successfully!")


✅ TF-IDF Vectorizer loaded successfully!


In [27]:
import joblib

# Load the trained vectorizer
tfidf_vectorizer = joblib.load("tfidf_vectorizer.pkl")
print("✅ Loaded TF-IDF Vectorizer with 5000 features")


✅ Loaded TF-IDF Vectorizer with 5000 features


In [28]:
def predict_fake_news(text):
    text_tfidf = tfidf_vectorizer.transform([text])  # Convert text to TF-IDF

    # Ensure it has the correct shape
    if text_tfidf.shape[1] != 5000:
        print(f"⚠️ Feature size mismatch! Expanding to 5000 features...")
        from scipy.sparse import hstack, csr_matrix
        text_tfidf = hstack([text_tfidf, csr_matrix((1, 5000 - text_tfidf.shape[1]))])  # Pad with zeros

    prediction = model.predict(text_tfidf)  # Predict
    return "Fake News" if prediction == 0 else "Real News"

# Test with a new article
new_article = "A new study claims that drinking lemon juice cures cancer."
result = predict_fake_news(new_article)
print(f"📰 Prediction: {result}")


⚠️ Feature size mismatch! Expanding to 5000 features...
📰 Prediction: Real News


In [29]:
test_articles = [
    "New research suggests that eating chocolate daily prevents heart disease.",
    "The WHO has confirmed a new virus outbreak in South America.",
    "Scientists claim that drinking 10 cups of coffee a day extends lifespan.",
    "A fake website is spreading misinformation about COVID-19 treatments.",
    "5G network causes COVID"
]
for article in test_articles:
    result = predict_fake_news(article)
    print(f"📰 {article}\n➡️ Prediction: {result}\n")


⚠️ Feature size mismatch! Expanding to 5000 features...
📰 New research suggests that eating chocolate daily prevents heart disease.
➡️ Prediction: Real News

⚠️ Feature size mismatch! Expanding to 5000 features...
📰 The WHO has confirmed a new virus outbreak in South America.
➡️ Prediction: Real News

⚠️ Feature size mismatch! Expanding to 5000 features...
📰 Scientists claim that drinking 10 cups of coffee a day extends lifespan.
➡️ Prediction: Real News

⚠️ Feature size mismatch! Expanding to 5000 features...
📰 A fake website is spreading misinformation about COVID-19 treatments.
➡️ Prediction: Real News

⚠️ Feature size mismatch! Expanding to 5000 features...
📰 5G network causes COVID
➡️ Prediction: Real News



In [30]:
!pip install flask twilio python-telegram-bot


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.1/676.1 kB 48.1 MB/s eta 0:00:00


In [31]:
pip install twilio flask


In [32]:
!pip install flask twilio python-telegram-bot


In [33]:
!pip install flask-ngrok


In [34]:
!pip install pyngrok


In [35]:
from flask import Flask, request, jsonify
from pyngrok import ngrok

app = Flask(__name__)

# Set your ngrok authtoken
ngrok.set_auth_token("2u1Zi4FX5ewrhTMVsXHvdaBYnEd_4KFKWvZGUkWySg5gN3RMx")  # Replace with your actual authtoken

# Start ngrok and get the public URL
public_url = ngrok.connect(5000)
print(f"🚀 Public URL: {public_url}")

🚀 Public URL: NgrokTunnel: "https://b7f2-34-125-230-107.ngrok-free.app" -> "http://localhost:5000"


In [36]:
!pip install twilio


In [37]:
from pyngrok import ngrok

# Stop previous ngrok instances (if any)
!pkill -f ngrok

# Start a new ngrok tunnel
public_url = ngrok.connect(5000)
print(f"🚀 Public URL for Twilio: {public_url}")


🚀 Public URL for Twilio: NgrokTunnel: "https://a829-34-125-230-107.ngrok-free.app" -> "http://localhost:5000"


In [38]:
!flask run --host=0.0.0.0 --port=5000


Usage: flask run [OPTIONS]
Try 'flask run --help' for help.

Error: Could not locate a Flask application. Use the 'flask --app' option, 'FLASK_APP' environment variable, or a 'wsgi.py' or 'app.py' file in the current directory.


In [39]:
from pyngrok import ngrok

public_url = ngrok.connect(5000)
print(f"🚀 Public URL for Twilio: {public_url}")


🚀 Public URL for Twilio: NgrokTunnel: "https://f913-34-125-230-107.ngrok-free.app" -> "http://localhost:5000"


In [40]:
!python app.py


python3: can't open file '/content/app.py': [Errno 2] No such file or directory


In [41]:
!ls


FakeHealth  FakeHealth.zip  sample_data  tfidf_vectorizer.pkl  xgboost_tfidf_model.pkl


In [43]:
!pip install flask flask-ngrok twilio joblib pyngrok


In [45]:
!curl -X POST https://xxxx.ngrok-free.app/whatsapp -d "Body=Test"  # Replace with your actual ngrok URL


<!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-Semibold-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://cdn.ngrok.com/static/fonts/ibm-plex-mono/IBMPlexMono-Tex

In [50]:
!pip install fastapi uvicorn joblib scikit-learn


In [60]:
!pip install python-multipart


In [61]:
from fastapi import FastAPI, Form, Request
from pydantic import BaseModel
import joblib
import uvicorn
from pyngrok import ngrok
from twilio.twiml.messaging_response import MessagingResponse
import os

# 🔹 Load Model and Vectorizer
model = joblib.load("xgboost_tfidf_model.pkl")
tfidf_vectorizer = joblib.load("tfidf_vectorizer.pkl")

# 🔹 Initialize FastAPI App
app = FastAPI()

# 🔹 Twilio Credentials (Replace with your credentials)
TWILIO_ACCOUNT_SID = "ACee8f4df84ad6e8300532d4bc75c2bd4b"
TWILIO_AUTH_TOKEN = "0710168256c62a7806660b58eac85126"

# 🔹 Start ngrok and get Public URL
public_url = ngrok.connect(5000).public_url
print(f"🚀 Public URL for Twilio: {public_url}/whatsapp")

# 🔹 Function to Predict Fake News
def predict_fake_news(title: str):
    text_tfidf = tfidf_vectorizer.transform([title])
    prediction = model.predict(text_tfidf)
    return "🛑 Fake News" if prediction == 0 else "✅ Real News"

# 🔹 WhatsApp Webhook for Twilio
@app.post("/whatsapp")
async def whatsapp_webhook(request: Request, Body: str = Form(...)):
    prediction = predict_fake_news(Body)

    # Twilio's Messaging Response
    response = MessagingResponse()
    response.message(f"📰 *Prediction:* {prediction}")

    return str(response)

# 🔹 Start FastAPI Server
uvicorn.run(app, host="0.0.0.0", port=5000)


🚀 Public URL for Twilio: https://e3a8-34-125-230-107.ngrok-free.app/whatsapp


INFO:     Started server process [186]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [186]
